# Skin lesion classification with transfer learning

Most of the code in this notebook was taken from https://github.com/falloutdurham/beginners-pytorch-deep-learning/. Here, it was tweaked to solve the sking lesion classification problem 

Please refer to the following book for more details:
Programming PyTorch for Deep Learning by Ian Pointer - Released September 2019, Publisher(s): O'Reilly Media, Inc., ISBN: 9781492045342

### Imports

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

folder_path = "/content/drive/My Drive/Skin_Lesions"

## Get data 

### Image transforms

In [0]:
im_size = 256

img_transforms = transforms.Compose([
    transforms.Resize((im_size,im_size)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

### Data loaders

In [0]:
train_data_path = folder_path + "/train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=img_transforms, is_valid_file=check_image)

test_data_path = folder_path + "/test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=img_transforms, is_valid_file=check_image)

val_data_path = folder_path + "/val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=img_transforms, is_valid_file=check_image)

batch_size = 64

rand_sampler_train = torch.utils.data.RandomSampler(train_data, replacement=False)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=rand_sampler_train)

rand_sampler_val = torch.utils.data.RandomSampler(val_data,replacement=False)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size,sampler=rand_sampler_val)

rand_sampler_test = torch.utils.data.RandomSampler(test_data, replacement=False) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size,sampler=rand_sampler_test)

## Load pre-trained model

In [0]:
transfer_model = models.resnet50(pretrained=True)

## Freeze parameters

Freezing everything except batch normalization layers

In [0]:
for name, param in transfer_model.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

## Replacing the classifier

In [0]:
transfer_model.fc = nn.Sequential(nn.Linear(transfer_model.fc.in_features,500),
nn.ReLU(),                                 
nn.Dropout(), nn.Linear(500,2))

# Training

## Training procedure

In [0]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [0]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [9]:
transfer_model.to(device)
optimizer = optim.Adam(transfer_model.parameters(), lr=0.001)

train(transfer_model, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=20, device=device)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0, Training Loss: 0.65, Validation Loss: 0.51, accuracy = 0.80
Epoch: 1, Training Loss: 0.43, Validation Loss: 0.47, accuracy = 0.80
Epoch: 2, Training Loss: 0.36, Validation Loss: 0.46, accuracy = 0.81
Epoch: 3, Training Loss: 0.27, Validation Loss: 0.56, accuracy = 0.81
Epoch: 4, Training Loss: 0.19, Validation Loss: 0.53, accuracy = 0.83
Epoch: 5, Training Loss: 0.10, Validation Loss: 0.74, accuracy = 0.83
Epoch: 6, Training Loss: 0.04, Validation Loss: 0.81, accuracy = 0.81
Epoch: 7, Training Loss: 0.02, Validation Loss: 0.97, accuracy = 0.81
Epoch: 8, Training Loss: 0.01, Validation Loss: 0.95, accuracy = 0.81
Epoch: 9, Training Loss: 0.00, Validation Loss: 1.01, accuracy = 0.81
Epoch: 10, Training Loss: 0.00, Validation Loss: 1.16, accuracy = 0.81
Epoch: 11, Training Loss: 0.00, Validation Loss: 1.17, accuracy = 0.81
Epoch: 12, Training Loss: 0.00, Validation Loss: 1.14, accuracy = 0.81
Epoch: 13, Training Loss: 0.00, Validation Loss: 1.12, accuracy = 0.81
Epoch: 14, Train

## Test

In [10]:
model = transfer_model
num_correct = 0 
num_examples = 0
valid_loss = 0.0
loss_fn = torch.nn.CrossEntropyLoss()

mag_total = 0
mag_right = 0
ben_total = 0
ben_right = 0


for batch in test_data_loader:
  inputs, targets = batch
  inputs = inputs.to(device)
  output = model(inputs)

  targets = targets.to(device)
  loss = loss_fn(output,targets) 
  valid_loss += loss.data.item() * inputs.size(0)

  classes = torch.max(F.softmax(output), dim=1)[1]
  
  for c,t in zip(classes,targets):
    if t == 1:
      mag_total += 1
      if c == 1:
        mag_right += 1
    else:
      ben_total += 1
      if c == 0:
         ben_right += 1

  correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
  num_correct += torch.sum(correct).item()
  num_examples += correct.shape[0]
valid_loss /= len(test_data_loader.dataset)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [11]:
print('Test loss: ' + str(valid_loss) + ' Accuracy: ' + str(num_correct / num_examples))
print('Acurracy malignant ' + str(mag_right/mag_total))
print('Acurracy benign ' + str(ben_right/ben_total))

Test loss: 1.1866410437604071 Accuracy: 0.8309859154929577
Acurracy malignant 0.42857142857142855
Acurracy benign 0.9298245614035088
